In [24]:
# import packages
import numpy as np
import pandas as pd
import warnings
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

In [2]:
# read the dataset
anime = pd.read_csv("anime_cleaned.csv")
anime.head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,...,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ...",24.0,2012.0
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi...",24.0,2007.0
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 101, 'type': 'manga...","TV Tokyo, Sotsu",NaN,Satelight,"Comedy, Magic, School, Shoujo","['#1: ""Minna no Tamago (みんなのたまご)"" by Shugo Cha...","['#1: ""Rottara Rottara (ロッタラ ロッタラ)"" by Buono! ...",24.0,2008.0
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,...,Fridays at Unknown,"{'Adaptation': [{'mal_id': 1581, 'type': 'mang...","Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"Comedy, Drama, Magic, Romance, Fantasy","['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ...",16.0,2002.0
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 9711, 'type': 'mang...","NHK, Shueisha",NaN,J.C.Staff,"Comedy, Drama, Romance, Shounen","['#1: ""Moshimo no Hanashi (もしもの話)"" by nano.RIP...","['#1: ""Pride on Everyday"" by Sphere (eps 1-13)...",24.0,2012.0


In [4]:
# select 'anime_id', 'title', 'type', 'episodes', 'score', 'scored_by', 'genre' col
anime_sub = anime.loc[:, ['anime_id', 'title', 'type', 'episodes', 'score', 'scored_by', 'genre']]

In [6]:
anime_sub.shape

(6668, 7)

In [7]:
anime_sub.head()

,anime_id,title,type,episodes,score,scored_by,genre
0,11013,Inu x Boku SS,TV,12,7.63,139250,"Comedy, Supernatural, Romance, Shounen"
1,2104,Seto no Hanayome,TV,26,7.89,91206,"Comedy, Parody, Romance, School, Shounen"
2,5262,Shugo Chara!! Doki,TV,51,7.55,37129,"Comedy, Magic, School, Shoujo"
3,721,Princess Tutu,TV,38,8.21,36501,"Comedy, Drama, Magic, Romance, Fantasy"
4,12365,Bakuman. 3rd Season,TV,25,8.67,107767,"Comedy, Drama, Romance, Shounen"


In [8]:
# only select tv show and movie
anime_tv_movie = anime_sub[(anime_sub['type'] == 'TV') | (anime_sub['type'] == 'Movie')]
print(anime_tv_movie.shape)

(3888, 7)


In [11]:
# only select famous anime, 75% percentile
quantile = anime_tv_movie['scored_by'].quantile(0.75)
anime_tv_movie = anime_tv_movie[(anime['scored_by'] >= quantile)]
anime_tv_movie.shape

(972, 7)

In [12]:
# read the rating csv
rating = pd.read_csv("rating.csv")
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [13]:
# Replacing missing rating in rating.csv with NaN
rating.loc[rating.rating == -1, 'rating'] = np.NaN
rating.head()

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN


In [15]:
# Create index for anime name
anime_index = pd.Series(anime_tv_movie.index, index=anime_tv_movie.title)
anime_index.head()

title
Inu x Boku SS          0
Seto no Hanayome       1
Shugo Chara!! Doki     2
Princess Tutu          3
Bakuman. 3rd Season    4
dtype: int64

In [16]:
# join the data: anime_tv_movie and rating
joined = anime_tv_movie.merge(rating, how='inner', on='anime_id')
joined.head()

,anime_id,title,type,episodes,score,scored_by,genre,user_id,rating
0,11013,Inu x Boku SS,TV,12,7.63,139250,"Comedy, Supernatural, Romance, Shounen",5,1.0
1,11013,Inu x Boku SS,TV,12,7.63,139250,"Comedy, Supernatural, Romance, Shounen",41,9.0
2,11013,Inu x Boku SS,TV,12,7.63,139250,"Comedy, Supernatural, Romance, Shounen",46,10.0
3,11013,Inu x Boku SS,TV,12,7.63,139250,"Comedy, Supernatural, Romance, Shounen",54,NaN
4,11013,Inu x Boku SS,TV,12,7.63,139250,"Comedy, Supernatural, Romance, Shounen",62,7.0


In [17]:
# Create a pivot table
joined = joined[['user_id', 'title', 'rating']]

# use 10000 users
joined = joined[(joined['user_id'] <= 10000)]

pivot = pd.pivot_table(joined, index='user_id', columns='title', values='rating')
pivot.head()

title,.hack//Sign,07-Ghost,11eyes,91 Days,Aa! Megami-sama! (TV),Absolute Duo,Accel World,Acchi Kocchi (TV),Afro Samurai,Afro Samurai: Resurrection,...,Zetman,Zetsuen no Tempest,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zoku Natsume Yuujinchou,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,ef: A Tale of Melodies.,ef: A Tale of Memories.,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,6.0,NaN,...,NaN,8.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,2.0,NaN
7,NaN,NaN,NaN,NaN,NaN,8.0,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
pivot.shape

(9360, 881)

In [19]:
# Drop all users that never rate an anime
pivot.dropna(axis=0, how='all', inplace=True)
pivot.head()

title,.hack//Sign,07-Ghost,11eyes,91 Days,Aa! Megami-sama! (TV),Absolute Duo,Accel World,Acchi Kocchi (TV),Afro Samurai,Afro Samurai: Resurrection,...,Zetman,Zetsuen no Tempest,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zoku Natsume Yuujinchou,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,ef: A Tale of Melodies.,ef: A Tale of Memories.,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,6.0,NaN,...,NaN,8.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,2.0,NaN
7,NaN,NaN,NaN,NaN,NaN,8.0,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
pivot.shape

(9360, 881)

In [21]:
# Center the mean around 0 (centered cosine / pearson)
pivot_norm = pivot.apply(lambda x: x - np.nanmean(x), axis=1)
pivot_norm.head()

title,.hack//Sign,07-Ghost,11eyes,91 Days,Aa! Megami-sama! (TV),Absolute Duo,Accel World,Acchi Kocchi (TV),Afro Samurai,Afro Samurai: Resurrection,...,Zetman,Zetsuen no Tempest,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zoku Natsume Yuujinchou,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,ef: A Tale of Melodies.,ef: A Tale of Memories.,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,-0.743243,NaN,-1.743243,NaN,...,NaN,0.256757,NaN,NaN,NaN,-0.743243,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,-2.305842,-1.305842,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.694158,NaN,NaN,NaN,-2.305842,NaN
7,NaN,NaN,NaN,NaN,NaN,0.461538,0.461538,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# apply User Based Collaborative Filtering
# fill NaN with 0
pivot_norm.fillna(0, inplace=True)
pivot_norm.head()

title,.hack//Sign,07-Ghost,11eyes,91 Days,Aa! Megami-sama! (TV),Absolute Duo,Accel World,Acchi Kocchi (TV),Afro Samurai,Afro Samurai: Resurrection,...,Zetman,Zetsuen no Tempest,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zoku Natsume Yuujinchou,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,ef: A Tale of Melodies.,ef: A Tale of Memories.,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,-0.743243,0.0,-1.743243,0.0,...,0.0,0.256757,0.0,0.0,0.000000,-0.743243,0.0,0.0,0.000000,0.0
5,0.0,0.0,0.0,0.0,0.0,-2.305842,-1.305842,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,2.694158,0.000000,0.0,0.0,-2.305842,0.0
7,0.0,0.0,0.0,0.0,0.0,0.461538,0.461538,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0


In [25]:
# calculate similar users
# convert into dataframe to make it easier
user_sim_df = pd.DataFrame(cosine_similarity(pivot_norm, pivot_norm), index=pivot_norm.index, columns=pivot_norm.index)
user_sim_df.head()

user_id,1,2,3,5,7,8,9,10,11,12,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,1.000000,0.073155,0.181504,0.018683,0.0,0.124471,0.082183,0.109459,...,-0.107795,-0.141579,0.005585,0.056867,0.052149,0.031906,-0.018372,0.074425,0.028931,0.065511
5,0.0,0.0,0.073155,1.000000,0.086039,0.016012,0.0,0.031973,0.033140,0.095139,...,0.005123,-0.053167,0.053327,0.014493,0.122310,0.183878,-0.017283,0.051711,0.029928,0.014486
7,0.0,0.0,0.181504,0.086039,1.000000,-0.013750,0.0,0.027670,-0.025430,0.068384,...,0.000000,0.000000,-0.002376,0.032270,0.092903,0.117040,0.017957,0.134479,0.051338,-0.016867


In [26]:
# define get_similar_user function
def get_similar_user(user_id):
    if user_id not in pivot_norm.index:
        return None, None
    else:
        sim_users = user_sim_df.sort_values(by=user_id, ascending=False).index[1:]
        sim_score = user_sim_df.sort_values(by=user_id, ascending=False).loc[:, user_id].tolist()[1:]
        return sim_users, sim_score

In [35]:
# print user similar results
def get_user_similar_result(user):
    users, score = get_similar_user(user)
    for x,y in zip(users[:15], score[:10]):
        print("User {} with similarity {} of user {}".format(x, y, user))



In [36]:
get_user_similar_result(3)

User 3028 with similarity 0.3702175443946056 of user 3
User 2986 with similarity 0.36708271550877813 of user 3
User 2411 with similarity 0.3614528416610143 of user 3
User 3681 with similarity 0.35428574548026326 of user 3
User 4481 with similarity 0.3504916236000324 of user 3
User 1966 with similarity 0.3492197359289317 of user 3
User 1606 with similarity 0.3480271422832077 of user 3
User 2038 with similarity 0.3377783704785195 of user 3
User 3990 with similarity 0.336605664258851 of user 3
User 656 with similarity 0.3294417663934893 of user 3


In [37]:
# Get Recommendation
def get_recommendation(user_id, n_anime=10):
    users, scores = get_similar_user(user_id)
    
    # there is no information for this user
    if users is None or scores is None:
        return None
    
    # only take 10 nearest users
    user_arr = np.array([x for x in users[:10]])
    sim_arr = np.array([x for x in scores[:10]])
    predicted_rating = np.array([])
    
    for anime_name in pivot_norm.columns:
        filtering = pivot_norm[anime_name].loc[user_arr] != 0.0  
        temp = np.dot(pivot[anime_name].loc[user_arr[filtering]], sim_arr[filtering]) / np.sum(sim_arr[filtering])
        predicted_rating = np.append(predicted_rating, temp)
    
    # don't recommend something that user has already rated
    temp = pd.DataFrame({'predicted':predicted_rating, 'name':pivot_norm.columns})
    filtering = (pivot_norm.loc[user_id] == 0.0)
    temp = temp.loc[filtering.values].sort_values(by='predicted', ascending=False)

    # recommend n_anime anime
    return anime.loc[anime_index.loc[temp.name[:n_anime]]]

In [38]:
# get_recommendation of user
get_recommendation(3)

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
3893,3712,Zero no Tsukaima: Princesses no Rondo,The Familiar of Zero: Rondo of Princesses,ゼロの使い魔 ～三美姫（プリンセッセ）の輪舞（ロンド）～,"Zero no Tsukaima: Princess no Rondo, Rondo of ...",https://myanimelist.cdn-dena.com/images/anime/...,TV,Light novel,12,Finished Airing,...,Mondays at Unknown,"{'Adaptation': [{'mal_id': 1154, 'type': 'mang...",NaN,Sentai Filmworks,J.C.Staff,"Action, Adventure, Harem, Comedy, Magic, Roman...","['""YOU&#039;RE THE ONE"" by ICHIKO']","['""Gomen ne (ゴメンネ)"" by Rie Kugimiya (釘宮理恵)']",24.0,2008.0
3673,4214,Rosario to Vampire Capu2,Rosario + Vampire Capu2,ロザリオとバンパイア Capu2,"Rosario + Vampire Season 2, Rosario and Vampir...",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,13,Finished Airing,...,Unknown,"{'Prequel': [{'mal_id': 2993, 'type': 'anime',...","GDH, Yomiko Advertising, Happinet Pictures, Sh...",Funimation,Gonzo,"Comedy, Ecchi, Fantasy, Harem, Romance, School...","['""DISCOTHEQUE"" by Nana Mizuki']","['""Trinity Cross"" by Nana Mizuki']",24.0,2008.0
639,28977,Gintama°,Gintama Season 4,銀魂°,Gintama&#039; (2015),https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,...,Wednesdays at 18:00 (JST),"{'Adaptation': [{'mal_id': 44, 'type': 'manga'...","TV Tokyo, Aniplex, Dentsu","Funimation, Crunchyroll",Bandai Namco Pictures,"Action, Comedy, Historical, Parody, Samurai, S...","['#1: ""DAY×DAY"" by BLUE ENCOUNT (eps 1-12)', '...","['#1: ""DESTINY"" by Negoto (eps 1-12)', '#2: ""S...",24.0,2015.0
1461,6201,Princess Lover!,NaN,プリンセスラバー!,Princess Lover!,https://myanimelist.cdn-dena.com/images/anime/...,TV,Visual novel,12,Finished Airing,...,Mondays at Unknown,"{'Adaptation': [{'mal_id': 15832, 'type': 'man...",Dream Force,NaN,GoHands,"Harem, Comedy, Ecchi, School","['""Princess Primp!"" by Miyuki Hashimoto']","['""S.S.D"" by yozuca*']",24.0,2009.0
4284,9969,Gintama&#039;,Gintama Season 2,銀魂&#039;,Gintama (2011),https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,...,Mondays at 18:00 (JST),"{'Adaptation': [{'mal_id': 44, 'type': 'manga'...","TV Tokyo, Aniplex, Dentsu, Trinity Sound, Mira...",NaN,Sunrise,"Action, Sci-Fi, Comedy, Historical, Parody, Sa...","['#1: ""Tougenkyou Alien (桃源郷エイリアン)"" by serial ...","['#1: ""Samurai Heart (Some Like It Hot!!)"" by ...",24.0,2011.0
2292,18689,Diamond no Ace,Ace of Diamond,ダイヤのA[エース],"Daiya no Ace, Ace of the Diamond, Dia no A",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,75,Finished Airing,...,Sundays at 08:30 (JST),"{'Adaptation': [{'mal_id': 10884, 'type': 'man...","TV Tokyo, Sotsu, Pony Canyon, Kodansha, AT-X, ...",NaN,"Production I.G, Madhouse","Comedy, School, Shounen, Sports","['#1: ""Go EXCEED!!"" by Tom-H@ck featuring Masa...","['#1: ""Seek Diamonds"" by Yoko Hikasa (eps 1-13...",24.0,2013.0
5710,31964,Boku no Hero Academia,My Hero Academia,僕のヒーローアカデミア,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,13,Finished Airing,...,Sundays at 17:00 (JST),"{'Adaptation': [{'mal_id': 75989, 'type': 'man...","Dentsu, Mainichi Broadcasting System, Movic, T...",Funimation,Bones,"Action, Comedy, School, Shounen, Super Power","['""The Day"" by Porno Graffitti']","['""HEROES"" by Brian the Sun']",24.0,2016.0
6331,15335,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Ei...,NaN,劇場版 銀魂 完結篇 万事屋よ永遠なれ,Gintama: The Final Chapter - Be Forever Yorozu...,https://myanimelist.cdn-dena.com/images/anime/...,Movie,Manga,1,Finished Airing,...,NaN,"{'Adaptation': [{'mal_id': 91209, 'type': 'man...","TV Tokyo, Aniplex, Dentsu, Bandai, Warner Bros...",NaN,Sunrise,"Action, Sci-Fi, Comedy, Historical, Parody, Sa...",[],"['#1: ""Genjou Destruction (現状ディストラクション)"" by SP...",110.0,2013.0
3147,918,Gintama,Gintama,銀魂,"Gin Tama, Silver Soul, Yorinuki Gintama-san",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,201,Finis

In [39]:
get_recommendation(5)

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
529,32281,Kimi no Na wa.,Your Name.,君の名は。,NaN,https://myanimelist.cdn-dena.com/images/anime/...,Movie,Original,1,Finished Airing,...,NaN,"{'Adaptation': [{'mal_id': 99314, 'type': 'man...","Kadokawa Shoten, Toho, Sound Team Don Juan, La...","Funimation, NYAV Post",CoMix Wave Films,"Supernatural, Drama, Romance, School","['""Yume Tourou (夢灯籠)"" by RADWIMPS']","['""Nandemonaiya (なんでもないや)"" by RADWIMPS']",106.0,2016.0
5402,1829,Gedo Senki,Tales from Earthsea,ゲド戦記,"Ged Senki, Tales of Ged, Ged&#039;s War Chroni...",https://myanimelist.cdn-dena.com/images/anime/...,Movie,Novel,1,Finished Airing,...,NaN,[],Toho,Walt Disney Studios,Studio Ghibli,"Adventure, Fantasy, Magic",[],"['""Toki no Uta (時の歌)"" by Aoi Teshima']",115.0,2006.0
1810,101,Air,Air,AIR,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Visual novel,12,Finished Airing,...,Fridays at 00:30 (JST),"{'Adaptation': [{'mal_id': 1604, 'type': 'mang...","Pony Canyon, TBS, Movic","ADV Films, Funimation",Kyoto Animation,"Slice of Life, Supernatural, Drama, Romance","['""Tori no Uta (鳥の詩)"" by Lia']","['#1: ""Farewell Song"" by Lia (eps 1-12)', '#2:...",24.0,2005.0
1072,585,Mimi wo Sumaseba,Whisper of the Heart,耳をすませば,"If You Listen Carefully, Ghibli Movie 10",https://myanimelist.cdn-dena.com/images/anime/...,Movie,Manga,1,Finished Airing,...,NaN,"{'Adaptation': [{'mal_id': 5228, 'type': 'mang...",Tokuma Shoten,"Walt Disney Studios, GKIDS",Studio Ghibli,"Slice of Life, Drama, Romance, Shoujo","['""Take Me Home, Country Roads"" by Olivia Newt...","['""Country Road (カントリー・ロード)"" by Youko Honna']",111.0,1995.0
6205,4565,Tengen Toppa Gurren Lagann Movie 2: Lagann-hen,Gurren Lagann The Movie: The Lights in the Sky...,天元突破グレンラガン 螺巌篇,Tengen Toppa Gurren Lagann Movie 2,https://myanimelist.cdn-dena.com/images/anime/...,Movie,Original,1,Finished Airing,...,NaN,"{'Alternative version': [{'mal_id': 2001, 'typ...",Aniplex,Aniplex of America,Gainax,"Action, Sci-Fi, Space, Super Power, Mecha",[],"['""Namida no Tane, Egao no Hana (涙の種、笑顔の花)"" by...",125.0,2009.0
1141,13125,Shinsekai yori,From the New World,新世界より,Shin Sekai yori,https://myanimelist.cdn-dena.com/images/anime/...,TV,Novel,25,Finished Airing,...,Saturdays at 00:30 (JST),"{'Adaptation': [{'mal_id': 41575, 'type': 'man...","Aniplex, TV Asahi, Pony Canyon",Sentai Filmworks,A-1 Pictures,"Sci-Fi, Mystery, Horror, Psychological, Supern...","['""Yuki ni Saku Hana (雪に咲く花)"" by Maria Akizuki...","['#1: ""Wareta Ringo (割れたリンゴ)"" by Saki Watanabe...",22.0,2012.0
1664,572,Kaze no Tani no Nausicaä,Nausicaä of the Valley of the Wind,風の谷のナウシカ,"Nausicaa of the Valley of the Wind, Kaze no Ta...",https://myanimelist.cdn-dena.com/images/anime/...,Movie,Manga,1,Finished Airing,...,NaN,"{'Adaptation': [{'mal_id': 651, 'type': 'manga...",Topcraft,"Walt Disney Studios, GKIDS","Studio Ghibli, Studio Hibari","Adventure, Fantasy","['""Kaze no Tani no Naushika (風の谷のナウシカ; Nausica...","['""Tori no Hito (鳥の人; Bird Person)"" by Joe His...",117.0,1984.0
1366,4181,Clannad: After Story,Clannad ~After Story~,CLANNAD〜AFTER STORY〜 クラナド アフターストーリー,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Visual novel,24,Finished Airing,...,Fridays at 01:59 (JST),"{'Alternative version': [{'mal_id': 1723, 'typ...","Pony Canyon, TBS, Rakuonsha, Animation Do",Sentai Filmworks,Kyoto Animation,"Slice of Life, Comedy, Supernatural, Drama, Ro...","['""Toki wo Kizamu Uta (時を刻む唄)"" by Lia']","['#1: ""TORCH"" by Lia (eps 1-21)', '#2: ""Chiisa...",24.0,2008.0
3663,19,Monster,Monster,モンスター,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,74,Finished Airing,...,Wednesdays at 00:40 (JST),"{'Adaptation': [{'mal_id': 1, 'type': 'manga',...","VAP, Shogakukan Productions, Nippon Television...",Viz Media,Madhouse,"Drama, Horror, Mystery, Police, Psychological,...","['